# Animal Mixer

Given two animal species, let's make a cross between them and visualize the resulting new animal.

## Imports

In [ ]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

## Initialization

In [ ]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

## System Messages

In [ ]:
system_message = "You are a famous zoologist-surgeon who makes crosses between animals, so new hybrid animals with mixed features of both original animals. "
system_message += "Given two animal species, you create a new species which is a hybrid between the two. Make sure it only has one head. "
system_message += "Describe the new species following the pattern: species X is a hybrid between species A and species B. "
system_message += "Species A and B are the two given species. Describe the new species briefly, in up to 3 sentences. "
system_message += "Always be accurate. If you don't know the answer, say so."

## Tools

In [ ]:
def get_animal_name(animal1, animal2):
    print(f"Tool get_animal_name called for the cross between {animal1} and {animal2}")
    first = len(animal1) // 2
    second = len(animal2) // 2
    name = animal1[:first] + animal2[second:]
    return name

In [ ]:
get_animal_name('capybara', 'elephant')

In [ ]:
animal_function = {
    "name": "get_animal_name",
    "description": "Get the name of the cross between the two given animals. Call this whenever you are given the names of the two original animals, for example when a user enters 'capybara' and 'elephant'",
    "parameters": {
        "type": "object",
        "properties": {
            "animal1": {
                "type": "string",
                "description": "the first original animal species",
            },
            "animal2": {
                "type": "string",
                "description": "the second original animal species",
            },
        },
        "required": ["animal1", "animal2"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": animal_function}]

## Image

In [ ]:
def artist(animal1, animal2):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a hybrid between {animal1} and {animal2}, with some features of {animal1} and some features of {animal2}, blended smoothly into a single hybrid animal, in photorealistic style. Make sure it only has one head and there is no text in the image.",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
image = artist("capybara", "elephant")
display(image)

## Audio

In [ ]:
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Well, hi there")

## Chat

In [ ]:
def chat(history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    image = None
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, animal1, animal2 = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        image = artist(animal1, animal2)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    animal1 = arguments.get('animal1')
    animal2 = arguments.get('animal2')
    animal_name = get_animal_name(animal1, animal2)
    response = {
        "role": "tool",
        "content": json.dumps({"animal1": animal1, "animal2": animal2, "animal_name": animal_name}),
        "tool_call_id": tool_call.id
    }
    return response, animal1, animal2

## Gradio UI

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)